In [21]:
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

In [9]:
data_full = pd.read_csv('..\data\homes.csv', index_col='Id')
data_test = pd.read_csv('..\data\home_test.csv' ,index_col='Id')


In [10]:
data_full.dropna(axis=0, subset=['SalePrice'], inplace=True)
y = data_full.SalePrice
data_full.drop(['SalePrice'], axis=1, inplace=True)

X = data_full.select_dtypes(exclude=['object'])
X_test = data_test.select_dtypes(exclude=['object'])


In [13]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size = 0.8, test_size = 0.2, random_state = 0)

In [14]:
X_train.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
Id,,,,,,,,,,,,,,,,,,,,,
619,20,90.0,11694,9,5,2007,2007,452.0,48,0,...,774,0,108,0,0,260,0,0,7,2007
871,20,60.0,6600,5,5,1962,1962,0.0,0,0,...,308,0,0,0,0,0,0,0,8,2009
93,30,80.0,13360,5,7,1921,2006,0.0,713,0,...,432,0,0,44,0,0,0,0,8,2009
818,20,NaN,13265,8,5,2002,2002,148.0,1218,0,...,857,150,59,0,0,0,0,0,7,2008
303,20,118.0,13704,7,5,2001,2002,150.0,0,0,...,843,468,81,0,0,0,0,0,1,2006


In [16]:
X_train.shape

(1168, 36)

In [20]:
missing_val_count = (X_train.isnull().sum())
missing_val_count[missing_val_count>0]

LotFrontage    212
MasVnrArea       6
GarageYrBlt     58
dtype: int64

In [22]:
def score_dataset (X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=100, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

*** 
## Drop columns with missing values

In [28]:
mis_col = []
for col in X_train.columns:
    if X_train[col].isnull().any():
        mis_col.append(col)
mis_col

['LotFrontage', 'MasVnrArea', 'GarageYrBlt']

In [29]:
# Fill in the lines below: drop columns in training and validation data
reduced_X_train = X_train.drop(mis_col, axis = 1)
reduced_X_valid = X_valid.drop(mis_col, axis = 1)

### MAE with drop column

In [30]:
print(score_dataset(reduced_X_train, reduced_X_valid, y_train, y_valid))

17837.82570776256


***
## Imputation

In [31]:
from sklearn.impute import SimpleImputer

my_imputer = SimpleImputer() 
imputed_X_train = pd.DataFrame(my_imputer.fit_transform(X_train))
imputed_X_valid = pd.DataFrame(my_imputer.transform(X_valid))

# Fill in the lines below: imputation removed column names; put them back
imputed_X_train.columns = X_train.columns
imputed_X_valid.columns = X_valid.columns

### MAE imputation

In [32]:
print(score_dataset(imputed_X_train, imputed_X_valid, y_train, y_valid))

18062.894611872147


***
## Final test

In [34]:
final_imputer = SimpleImputer(strategy='median')
final_X_train = pd.DataFrame(final_imputer.fit_transform(X_train))
final_X_valid = pd.DataFrame(final_imputer.transform(X_valid))

# Imputation removed column names; put them back
final_X_train.columns = X_train.columns
final_X_valid.columns = X_valid.columns

In [35]:
# Define and fit model
model = RandomForestRegressor(n_estimators=100, random_state=0)
model.fit(final_X_train, y_train)

# Get validation predictions and MAE
preds_valid = model.predict(final_X_valid)
print("MAE (Your approach):")
print(mean_absolute_error(y_valid, preds_valid))

MAE (Your approach):
17791.59899543379


In [36]:
# Fill in the line below: preprocess test data
final_X_test = pd.DataFrame(final_imputer.transform(X_test))

# Fill in the line below: get test predictions
preds_test = model.predict(final_X_test)

In [37]:
output = pd.DataFrame({'Id': X_test.index,
                       'SalePrice': preds_test})
output

,Id,SalePrice
0,1461,125985.50
1,1462,154599.50
2,1463,180070.24
3,1464,183544.50
4,1465,197549.92
...,...,...
1454,2915,87437.12
1455,2916,86994.50
1456,2917,154751.44
1457,2918,107387.00
